In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
import random

import tensorflow as tf
import tensorflow_io as tfio
import os

In [5]:
DATA_DIR = './dataset/all/'


EMOTION_DICT_RAVDEES = {
    "01": "neutral",
    "02": "calm",
    "03": "happy",
    "04": "sad",
    "05": "angry",
    "06": "fearful",
    "07": "disgust",
    "08": "surprised",
}

def decompose_label(file_path: str):
    return label_to_int[file_path.split("-")[2]]

file_path_list = os.listdir(DATA_DIR)
label_to_int = dict({(key, i) for i, key in enumerate(EMOTION_DICT_RAVDEES.keys())})
labels = [decompose_label(file_path) for file_path in file_path_list]

In [8]:
# Potential class weights ? 
# total = np.sum(train_df['label'].value_counts())
# class_weights = [1-(x/total) for x in train_df['label'].value_counts()]
# class_weights = np.array(class_weights)/6
# class_weights

In [ ]:
# SR = 22050 for ravdees
import tensorflow_io as tfio
file_contents = tf.io.read_file(os.path.join(DATA_DIR, file_path_list[0]))
print(file_contents)

In [ ]:
def load_wav(file_path, sr=22050):
    pre_emp = 0.97
    file_contents = tf.io.read_file(file_path)

    # Default SR is 22050, putting desired_samples to 66150 to get 3 second sample
    wav, sr = tf.audio.decode_wav(
        file_contents, desired_channels=1, desired_samples=66150
    )

    wav = tf.squeeze(wav, axis=-1)

    # Apply preamp if needed
    wav = tf.experimental.numpy.append(wav[0], wav[1:] - pre_emp * wav[:-1])

    return wav


def get_mfcc(wav, sr=22050):
    stft_out = tf.signal.stft(
        wav, 400, 160, window_fn=tf.signal.hamming_window, pad_end=False
    )
    num_spectrogram_bins = tf.shape(stft_out)[-1]
    stft_abs = tf.abs(stft_out)
    lower_edge_hz, upper_edge_hz = 20.0, 8000.0
    num_mel_bins = 64
    linear_to_mel_weight_matrix = tf.signal.linear_to_mel_weight_matrix(
        num_mel_bins, num_spectrogram_bins, 16000, lower_edge_hz, upper_edge_hz
    )
    mel_spectrograms = tf.tensordot(stft_abs, linear_to_mel_weight_matrix, 1)
    mel_spectrograms.set_shape(stft_abs.shape[:-1].concatenate(linear_to_mel_weight_matrix.shape[-1:]))

    log_mel_spectrograms = tf.math.log(mel_spectrograms + 1e-6)
    log_mel_d1 = log_mel_spectrograms - tf.roll(log_mel_spectrograms, -1, axis = 0)
    log_mel_d2 = log_mel_d1 - tf.roll(log_mel_d1, -1, axis = 0)

    log_mel_three_channel = tf.stack([log_mel_spectrograms, log_mel_d1, log_mel_d2], axis = -1)
    
    framed_log_mels = tf.signal.frame(log_mel_three_channel, frame_length=64, frame_step=32, pad_end=False, axis = 0)

    return framed_log_mels

In [16]:
import pandas as pd
filenames = os.listdir(DATA_DIR)
transcript_dict = {
    "01" : "Kids are talking by the door",
    "02" : "Dogs are sitting by the door"
}

transcript_list = [transcript_dict[x.split("-")[5]] for x in filenames]

dummy_df = pd.DataFrame(zip(filenames, transcript_list),columns=['filename', 'transcript'],)
dummy_df.to_csv("filename_transcript.csv",index=False)

In [19]:
# Get transcripts and stuff, write util functions to get transcripts from the filenames
# For RAVDEES there are only two statements. For other datasets there may be more. Better to go with something that can accept a csv
# with filename and transcript, via a csv maybe

def parse_filename_transcript_csv(csv_path):
    df = pd.read_csv(csv_path)
    file_name_list = np.array(df['filename'])
    transcript_list = np.array(df['transcript'])

    return (file_name_list, transcript_list)

file_name_list, transcripts = parse_filename_transcript_csv("./filename_transcript.csv")

In [20]:
from sklearn.utils import shuffle
file_name_list, transcripts, labels = shuffle(file_name_list, transcripts, labels)
train_fps, val_fps, train_transcripts, val_transcripts, train_labels, val_labels = train_test_split(filepaths, transcripts, labels, test_size = 0.1)